In [ ]:
PUBG Finish Placement Prediction: playground

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('train.csv')
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,2860d3f9ee4d4b,46f1b1c3bf0dae,25ade93a241f39,0,0,0.0,0,0,0,80,...,0,0.0,0,0.0,0,0,257.0,4,0,0.167487
1,0b88c47e54cdcc,5e92598b08bcb1,95125f5e05c653,0,2,359.0,4,0,0,5,...,1,262.7,0,0.0,0,0,1025.0,5,1607,0.610492
2,8f086d61515bc4,ed4cfbc560f490,8a84da96d077c3,0,0,223.5,1,0,0,15,...,0,0.0,0,0.0,0,0,712.8,2,1736,0.666206
3,7c037ca62a38a4,d2ff4830704e19,a139eac809980a,0,0,128.1,0,0,0,50,...,0,0.0,0,0.0,0,0,2738.0,4,1476,0.722956
4,82f08cde238c60,7c5002766d8c7a,e9d203a36b1d0a,0,0,200.0,2,0,0,36,...,0,0.0,0,0.0,0,0,129.4,2,0,0.204095


In [4]:
test = pd.read_csv('test.csv')


In [5]:
train_x = train.drop(["winPlacePerc"],axis = 1)
train_y = train["winPlacePerc"]

In [6]:
test_x = test.copy()

In [7]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7134 entries, 0 to 7133
Data columns (total 28 columns):
Id                 7134 non-null object
groupId            7134 non-null object
matchId            7134 non-null object
assists            7134 non-null int64
boosts             7134 non-null int64
damageDealt        7134 non-null float64
DBNOs              7134 non-null int64
headshotKills      7134 non-null int64
heals              7134 non-null int64
killPlace          7134 non-null int64
killPoints         7134 non-null int64
kills              7134 non-null int64
killStreaks        7134 non-null int64
longestKill        7134 non-null float64
matchDuration      7134 non-null int64
matchType          7134 non-null object
maxPlace           7134 non-null int64
numGroups          7134 non-null int64
rankPoints         7134 non-null int64
revives            7134 non-null int64
rideDistance       7134 non-null float64
roadKills          7134 non-null int64
swimDistance       7134 n

In [11]:
all_x = pd.concat([train_x,test_x])
all_x = pd.get_dummies(all_x)

In [12]:
train_x = all_x.iloc[:train_x.shape[0],:].reset_index(drop=True)
test_x = all_x.iloc[train_x.shape[0]:,:].reset_index(drop=True)

In [20]:
train_x.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,2860d3f9ee4d4b,46f1b1c3bf0dae,25ade93a241f39,0,0,0.0,0,0,0,80,...,1496,0,0.0,0,0.0,0,0,257.0,4,0
1,0b88c47e54cdcc,5e92598b08bcb1,95125f5e05c653,0,2,359.0,4,0,0,5,...,-1,1,262.7,0,0.0,0,0,1025.0,5,1607
2,8f086d61515bc4,ed4cfbc560f490,8a84da96d077c3,0,0,223.5,1,0,0,15,...,-1,0,0.0,0,0.0,0,0,712.8,2,1736
3,7c037ca62a38a4,d2ff4830704e19,a139eac809980a,0,0,128.1,0,0,0,50,...,-1,0,0.0,0,0.0,0,0,2738.0,4,1476
4,82f08cde238c60,7c5002766d8c7a,e9d203a36b1d0a,0,0,200.0,2,0,0,36,...,1479,0,0.0,0,0.0,0,0,129.4,2,0


In [ ]:
PUBG Finish Placement Prediction: playground

In [9]:
 nplayers = train_x.groupby('matchId')['matchId'].transform('count')
train_x['killPlaceScall'] = train_x['killPlace'] / nplayers
train_x['damageRate'] = train_x['kills']/(0.01*train_x['damageDealt'])
train_x['headshotRate'] = train_x['headshotKills']/train_x['kills']
train_x['killStreakRate'] = train_x['killStreaks']/train_x['kills']
train_x['healthItems'] = train_x['heals'] + train_x['boosts']
train_x['healsOnKill'] = train_x['healthItems']/train_x['kills']
train_x['sniper'] = train_x['longestKill']/100*train_x['weaponsAcquired']
train_x['totalDistance'] = train_x['rideDistance'] + train_x["walkDistance"] + train_x["swimDistance"]
train_x['totalDistancePen'] = train_x['rideDistance']/5 + train_x["walkDistance"] + train_x["swimDistance"]*10
train_x['killsPerRoadDist'] = train_x['roadKills'] / (train_x['rideDistance']+1)
train_x['killsPerWalkDist'] = (train_x['kills']-train_x['roadKills']) / (train_x['walkDistance']+1)
train_x['killsPerDist'] = train_x['kills']/(train_x['totalDistance']+1)
train_x['distance_over_weapons'] = train_x['totalDistance'] / train_x['weaponsAcquired']
train_x['walkDistance_over_heals'] = train_x['walkDistance']/100/train_x['heals']
train_x["skill"] = train_x["headshotKills"] + train_x["roadKills"] - train_x['teamKills'] 
train_x.fillna(0,inplace=True)
train_x.replace(np.inf, 0, inplace=True)

In [10]:
train_x.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,healsOnKill,sniper,totalDistance,totalDistancePen,killsPerRoadDist,killsPerWalkDist,killsPerDist,distance_over_weapons,walkDistance_over_heals,skill
0,2860d3f9ee4d4b,46f1b1c3bf0dae,25ade93a241f39,0,0,0.0,0,0,0,80,...,0.0,0.0000,257.0,257.00,0.0,0.000000,0.000000,64.25,0.0,0
1,0b88c47e54cdcc,5e92598b08bcb1,95125f5e05c653,0,2,359.0,4,0,0,5,...,0.5,1.7850,1287.7,1077.54,0.0,0.003899,0.003104,257.54,0.0,0
2,8f086d61515bc4,ed4cfbc560f490,8a84da96d077c3,0,0,223.5,1,0,0,15,...,0.0,0.5478,712.8,712.80,0.0,0.002802,0.002802,356.40,0.0,0
3,7c037ca62a38a4,d2ff4830704e19,a139eac809980a,0,0,128.1,0,0,0,50,...,0.0,0.0000,2738.0,2738.00,0.0,0.000000,0.000000,684.50,0.0,0
4,82f08cde238c60,7c5002766d8c7a,e9d203a36b1d0a,0,0,200.0,2,0,0,36,...,0.0,0.5150,129.4,129.40,0.0,0.007669,0.007669,64.70,0.0,0


In [13]:
train_x['groupId'] = train_x['groupId'].astype('category')
train_x['matchId'] = train_x['matchId'].astype('category')

# Get category coding for groupId and matchID
train_x['groupId_cat'] = train_x['groupId'].cat.codes
train_x['matchId_cat'] = train_x['matchId'].cat.codes

# Get rid of old columns
train_x.drop(columns=['groupId', 'matchId'], inplace=True)

# Lets take a look at our newly created features
train_x[['groupId_cat', 'matchId_cat']].head()

,groupId_cat,matchId_cat
0,904,11
1,1183,36
2,3010,33
3,2679,41
4,1571,66


In [14]:
train_x.head()

,Id,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,...,totalDistance,totalDistancePen,killsPerRoadDist,killsPerWalkDist,killsPerDist,distance_over_weapons,walkDistance_over_heals,skill,groupId_cat,matchId_cat
0,2860d3f9ee4d4b,0,0,0.0,0,0,0,80,0,0,...,257.0,257.00,0.0,0.000000,0.000000,64.25,0.0,0,904,11
1,0b88c47e54cdcc,0,2,359.0,4,0,0,5,1387,4,...,1287.7,1077.54,0.0,0.003899,0.003104,257.54,0.0,0,1183,36
2,8f086d61515bc4,0,0,223.5,1,0,0,15,1673,2,...,712.8,712.80,0.0,0.002802,0.002802,356.40,0.0,0,3010,33
3,7c037ca62a38a4,0,0,128.1,0,0,0,50,1479,0,...,2738.0,2738.00,0.0,0.000000,0.000000,684.50,0.0,0,2679,41
4,82f08cde238c60,0,0,200.0,2,0,0,36,0,1,...,129.4,129.40,0.0,0.007669,0.007669,64.70,0.0,0,1571,66


In [18]:
train_x.drop(columns = ['Id'], inplace=True)

In [19]:
train_x.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,totalDistance,totalDistancePen,killsPerRoadDist,killsPerWalkDist,killsPerDist,distance_over_weapons,walkDistance_over_heals,skill,groupId_cat,matchId_cat
0,0,0,0.0,0,0,0,80,0,0,0,...,257.0,257.00,0.0,0.000000,0.000000,64.25,0.0,0,904,11
1,0,2,359.0,4,0,0,5,1387,4,2,...,1287.7,1077.54,0.0,0.003899,0.003104,257.54,0.0,0,1183,36
2,0,0,223.5,1,0,0,15,1673,2,1,...,712.8,712.80,0.0,0.002802,0.002802,356.40,0.0,0,3010,33
3,0,0,128.1,0,0,0,50,1479,0,0,...,2738.0,2738.00,0.0,0.000000,0.000000,684.50,0.0,0,2679,41
4,0,0,200.0,2,0,0,36,0,1,1,...,129.4,129.40,0.0,0.007669,0.007669,64.70,0.0,0,1571,66


In [20]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7134 entries, 0 to 7133
Data columns (total 42 columns):
assists                    7134 non-null int64
boosts                     7134 non-null int64
damageDealt                7134 non-null float64
DBNOs                      7134 non-null int64
headshotKills              7134 non-null int64
heals                      7134 non-null int64
killPlace                  7134 non-null int64
killPoints                 7134 non-null int64
kills                      7134 non-null int64
killStreaks                7134 non-null int64
longestKill                7134 non-null float64
matchDuration              7134 non-null int64
matchType                  7134 non-null object
maxPlace                   7134 non-null int64
numGroups                  7134 non-null int64
rankPoints                 7134 non-null int64
revives                    7134 non-null int64
rideDistance               7134 non-null float64
roadKills                  7134 non-null

In [21]:
train_x = pd.get_dummies(train_x,columns = ["matchType"])


In [22]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7134 entries, 0 to 7133
Data columns (total 48 columns):
assists                    7134 non-null int64
boosts                     7134 non-null int64
damageDealt                7134 non-null float64
DBNOs                      7134 non-null int64
headshotKills              7134 non-null int64
heals                      7134 non-null int64
killPlace                  7134 non-null int64
killPoints                 7134 non-null int64
kills                      7134 non-null int64
killStreaks                7134 non-null int64
longestKill                7134 non-null float64
matchDuration              7134 non-null int64
maxPlace                   7134 non-null int64
numGroups                  7134 non-null int64
rankPoints                 7134 non-null int64
revives                    7134 non-null int64
rideDistance               7134 non-null float64
roadKills                  7134 non-null int64
swimDistance               7134 non-null 

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_holdout, y_train, y_holdout = train_test_split(train_x, train_y, test_size=0.2, random_state=666)

In [24]:
from sklearn.model_selection import cross_val_score
import sklearn.metrics
from sklearn.model_selection import GridSearchCV

In [25]:
import lightgbm as lgb

In [26]:
lgtrain = lgb.Dataset(X_train, label=y_train.reset_index(drop=True))
res = lgb.cv({'metric': 'mae'},lgtrain, nfold=5,stratified=False,seed=666)
print("Mean score:",res['l1-mean'][-1])

Mean score: 0.05031095591255338


In [27]:
gridParams = {
    'num_leaves': [30,50,100], 'max_depth': [-1,8,15], 
    'min_data_in_leaf': [100,300,500], 'max_bin': [250,500], 
    'lambda_l1': [0.01], 'num_iterations': [5], 
    'nthread': [4], 'seed': [666],
    'learning_rate': [0.05], 'metric': ['mae'],
    "bagging_fraction" : [0.7], "bagging_seed" : [0], "colsample_bytree" : [0.7]
    }
model = lgb.LGBMRegressor()
grid = GridSearchCV(model, gridParams,
                    verbose=1,
                    cv=5)

In [28]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7134 entries, 0 to 7133
Data columns (total 48 columns):
assists                    7134 non-null int64
boosts                     7134 non-null int64
damageDealt                7134 non-null float64
DBNOs                      7134 non-null int64
headshotKills              7134 non-null int64
heals                      7134 non-null int64
killPlace                  7134 non-null int64
killPoints                 7134 non-null int64
kills                      7134 non-null int64
killStreaks                7134 non-null int64
longestKill                7134 non-null float64
matchDuration              7134 non-null int64
maxPlace                   7134 non-null int64
numGroups                  7134 non-null int64
rankPoints                 7134 non-null int64
revives                    7134 non-null int64
rideDistance               7134 non-null float64
roadKills                  7134 non-null int64
swimDistance               7134 non-null 

In [29]:
grid.fit(X_train.iloc[:5000,:], y_train.iloc[:5000])

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed:   17.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True...
             param_grid={'bagging_fraction': [0.7], 'bagging_seed': [0],
                         'colsample_bytree': [0.7], 'lambda_l1': [0.01],
                         'learning_rate': [0.05], 'max_bin': [250, 500],
                         'max_depth': [-1, 8, 15], 'metric': ['mae'],
                         'min_data_in_leaf': [100, 300, 500], 'nthr

In [30]:
print("Best params:", grid.best_params_)
print("\nBest score:", grid.best_score_)
params = grid.best_params_

Best params: {'bagging_fraction': 0.7, 'bagging_seed': 0, 'colsample_bytree': 0.7, 'lambda_l1': 0.01, 'learning_rate': 0.05, 'max_bin': 500, 'max_depth': -1, 'metric': 'mae', 'min_data_in_leaf': 100, 'nthread': 4, 'num_iterations': 5, 'num_leaves': 50, 'seed': 666}

Best score: 0.3465547000872916


In [32]:
%%time
lgtrain = lgb.Dataset(X_train, label=y_train)
lgval = lgb.Dataset(X_holdout, label=y_holdout)

params['num_iterations'] = 5000
model = lgb.train(params, lgtrain, valid_sets=[lgtrain, lgval], early_stopping_rounds=200, verbose_eval=1000)

Training until validation scores don't improve for 200 rounds
[1000]	training's l1: 0.0154816	valid_1's l1: 0.0437813
[2000]	training's l1: 0.00785757	valid_1's l1: 0.0433157
Early stopping, best iteration is:
[2498]	training's l1: 0.00587442	valid_1's l1: 0.0432658
Wall time: 6.16 s


In [36]:
 nplayers = test_x.groupby('matchId')['matchId'].transform('count')
test_x['killPlaceScall'] = test_x['killPlace'] / nplayers
test_x['damageRate'] = test_x['kills']/(0.01*test_x['damageDealt'])
test_x['headshotRate'] = test_x['headshotKills']/test_x['kills']
test_x['killStreakRate'] = test_x['killStreaks']/test_x['kills']
test_x['healthItems'] = test_x['heals'] + test_x['boosts']
test_x['healsOnKill'] = test_x['healthItems']/train_x['kills']
test_x['sniper'] = test_x['longestKill']/100*train_x['weaponsAcquired']
test_x['totalDistance'] = test_x['rideDistance'] + test_x["walkDistance"] + test_x["swimDistance"]
test_x['totalDistancePen'] = test_x['rideDistance']/5 + test_x["walkDistance"] + test_x["swimDistance"]*10
test_x['killsPerRoadDist'] = test_x['roadKills'] / (test_x['rideDistance']+1)
test_x['killsPerWalkDist'] = (test_x['kills']-test_x['roadKills']) / (test_x['walkDistance']+1)
test_x['killsPerDist'] = test_x['kills']/(test_x['totalDistance']+1)
test_x['distance_over_weapons'] = test_x['totalDistance'] / test_x['weaponsAcquired']
test_x['walkDistance_over_heals'] = test_x['walkDistance']/100/test_x['heals']
test_x["skill"] = test_x["headshotKills"] + test_x["roadKills"] - test_x['teamKills'] 
test_x.fillna(0,inplace=True)
test_x.replace(np.inf, 0, inplace=True)

In [37]:
test_x['groupId'] = test_x['groupId'].astype('category')
test_x['matchId'] = test_x['matchId'].astype('category')

# Get category coding for groupId and matchID
test_x['groupId_cat'] = test_x['groupId'].cat.codes
test_x['matchId_cat'] = test_x['matchId'].cat.codes

# Get rid of old columns
test_x.drop(columns=['groupId', 'matchId'], inplace=True)

# Lets take a look at our newly created features
test_x[['groupId_cat', 'matchId_cat']].head()

,groupId_cat,matchId_cat
0,717,9
1,825,13
2,717,9
3,424,16
4,89,0


In [38]:
test_x.drop(columns = ['Id'], inplace=True)

In [39]:
test_x = pd.get_dummies(test_x,columns = ["matchType"])

In [42]:
pred_test = model.predict(test_x, num_iteration=model.best_iteration)
pred_df = pd.DataFrame({'winPlacePerc' : abs(pred_test)})

pred_df.to_csv("submission3.csv", index=False)